In [1]:
from nltk.tokenize import word_tokenize
import pandas as pd
import re
from text_to_num import text2num
from collections import Counter
from sklearn.model_selection import train_test_split

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# PROCESO DE TOKENIZADO

## PROCESO DE TOKENIZADO PARA DATOS 20N

In [51]:
#Lectura de datos del paso anterior
df_20n = pd.read_csv("20N_unified.csv")

In [52]:
#Selecciónn de columna correspondiente a las oraciones
body_20n = df_20n[['body']]

### Funciones básicas de tokenizado

La función normalize_text se encarga de convertir a todas las letras de las oraciones en minúsculas, y posteriormente elimina todos los caracteres especiales como las comas y los puntos y separa cada signo de puntuación con un espacio

La función words_to_numbers maneja un caso excepcional para convertir palabras que representen números como dígitos, para ello utiliza la librería text2num para convertir las palabras que sean necesarias a lo largo del corpus

La función replace_digits se encarga de convertir todos los digitos del corpus en el caracter NUM

In [53]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

def words_to_numbers(token):
    try:
        return str(text2num(token,'en'))
    except ValueError:
        return token

def replace_digits(token):
    if token.isdigit():
        return 'NUM'
    return token


En primer lugar, se aplica la normalización antes de realizar el tokenizado de palabras

In [54]:
body_20n['normalized'] = body_20n['body'].apply(normalize_text)

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/3604201721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_20n['normalized'] = body_20n['body'].apply(normalize_text)


Posteriormente, se hace una tokenización con la librería word_tokenize de nltk

In [57]:
body_20n['tokenized'] = body_20n['normalized'].apply(word_tokenize)

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/814313017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_20n['tokenized'] = body_20n['normalized'].apply(word_tokenize)


Luego, se realiza el reemplazo de palabras que representen dígitos en dígitos, esto con el objetivo de facilitar la conversión de dígitos en caracteres NUM en el siguiente paso

In [58]:
body_20n['digits'] = body_20n['tokenized'].apply(lambda tokens: [words_to_numbers(token) for token in tokens])

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/282450380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_20n['digits'] = body_20n['tokenized'].apply(lambda tokens: [words_to_numbers(token) for token in tokens])


Luego de haber hecho el reemplazo de palabras, se hace el reemplazo de todos los dígitos en un caracter denominado NUM

In [59]:
body_20n['replacement'] = body_20n['digits'].apply(lambda tokens: [replace_digits(token) for token in tokens])

Se añade un <s> al comienzo de cada oración y se añade un </S> al término de cada oración

In [60]:
body_20n['sentence'] = body_20n['replacement'].apply(lambda tokens: ['<s>'] + tokens + ['</s>'])

Por último, se realiza un conteo de todos los tokens en el corpus, si un token sólo aparece una vez, entonces se reemplaza por el caracter <UNK>

In [76]:
all_tokens = [token for tokens in body_20n['sentence'] for token in tokens]
token_freq = Counter(all_tokens)

def replace_unique_tokens(tokens, freq_dict):
    return [token if freq_dict[token] > 1 else '<UNK>' for token in tokens]


Reemplazo del token en el dataframe

In [77]:
body_20n['unique'] = body_20n['sentence'].apply(lambda tokens: replace_unique_tokens(tokens, token_freq))

Exportación de resultados obtenidos para realizar la división de datos

In [83]:
results_20n = pd.DataFrame(body_20n['unique'].apply(lambda tokens: ' '.join(tokens)))

In [124]:
results_20n.to_csv("20n_tokenized.csv",index=False)

In [78]:
body_20n

,body,normalized,tokenized,digits,replacement,sentence,unique
0,Archive-name: atheism/resources Alt-atheism-ar...,archive name atheism resources alt atheism ar...,"[archive, name, atheism, resources, alt, athei...","[archive, name, atheism, resources, alt, athei...","[archive, name, atheism, resources, alt, athei...","[<s>, archive, name, atheism, resources, alt, ...","[<s>, archive, name, atheism, resources, alt, ..."
1,Archive-name: atheism/introduction Alt-atheism...,archive name atheism introduction alt atheism...,"[archive, name, atheism, introduction, alt, at...","[archive, name, atheism, introduction, alt, at...","[archive, name, atheism, introduction, alt, at...","[<s>, archive, name, atheism, introduction, al...","[<s>, archive, name, atheism, introduction, al..."
2,In article <65974@mimsy.umd.edu> mangoe@cs.umd...,in article 65974 mimsy umd edu mangoe cs umd...,"[in, article, 65974, mimsy, umd, edu, mangoe, ...","[in, article, 65974, mimsy, umd, edu, mangoe, ...","[in, article, NUM, mimsy, umd, edu, mangoe, cs...","[<s>, in, article, NUM, mimsy, umd, edu, mango...","[<s>, in, article, NUM, mimsy, umd, edu, mango..."
3,dmn@kepler.unh.edu (...until kings become phil...,dmn kepler unh edu until kings become phil...,"[dmn, kepler, unh, edu, until, kings, become, ...","[dmn, kepler, unh, edu, until, kings, become, ...","[dmn, kepler, unh, edu, until, kings, become, ...","[<s>, dmn, kepler, unh, edu, until, kings, bec...","[<s>, dmn, kepler, unh, edu, until, kings, bec..."
4,In article <N4HY.93Apr5120934@harder.ccr-p.ida...,in article n4hy 93apr5120934 harder ccr p ida...,"[in, article, n4hy, 93apr5120934, harder, ccr,...","[in, article, n4hy, 93apr5120934, harder, ccr,...","[in, article, n4hy, 93apr5120934, harder, ccr,...","[<s>, in, article, n4hy, 93apr5120934, harder,...","[<s>, in, article, n4hy, <UNK>, harder, ccr, p..."
...,...,...,...,...,...,...,...
18789,pboxrud@magnus.acs.ohio-state.edu (Paul D Boxr...,pboxrud magnus acs ohio state edu paul d boxr...,"[pboxrud, magnus, acs, ohio, state, edu, paul,...","[pboxrud, magnus, acs, ohio, state, edu, paul,...","[pboxrud, magnus, acs, ohio, state, edu, paul,...","[<s>, pboxrud, magnus, acs, ohio, state, edu, ...","[<s>, <UNK>, magnus, acs, ohio, state, edu, pa..."
18790,"In article <1993Apr23.111105.7703@ifi.uio.no>,...",in article 1993apr23 111105 7703 ifi uio no ...,"[in, article, 1993apr23, 111105, 7703, ifi, ui...","[in, article, 1993apr23, 111105, 7703, ifi, ui...","[in, article, 1993apr23, NUM, NUM, ifi, uio, n...","[<s>, in, article, 1993apr23, NUM, NUM, ifi, u...","[<s>, in, article, 1993apr23, NUM, NUM, ifi, u..."
18791,In article <1rc1f3INN7rl@emx.cc.utexas.edu> bi...,in article 1rc1f3inn7rl emx cc utexas edu bi...,"[in, article, 1rc1f3inn7rl, emx, cc, utexas, e...","[in, article, 1rc1f3inn7rl, emx, cc, utexas, e...","[in, article, 1rc1f3inn7rl, emx, cc, utexas, e...","[<s>, in, article, 1rc1f3inn7rl, emx, cc, utex...","[<s>, in, article, <UNK>, emx, cc, utexas, edu..."
18792,In article <1993Apr26.231845.13843@digi.lonest...,in article 1993apr26 231845 13843 digi lonest...,"[in, article, 1993apr26, 231845, 13843, digi, ...","[in, article, 1993apr26, 231845, 13843, digi, ...","[in, article, 1993apr26, NUM, NUM, digi, lones...","[<s>, in, article, 1993apr26, NUM, NUM, digi, ...","[<s>, in, article, 1993apr26, NUM, NUM, digi, ..."


## PROCESO DE TOKENIZADO PARA DATOS BAC

Se utilizó el mismo proceso de tokenización de los datos 20N, en este caso sólo se utilizo el llamado de funciones puesto a que las funciones ya fueron creadas para los datos anteriores

In [107]:
df_bac = pd.read_csv("bac_unified.csv")

In [108]:
df_bac

,id,gender,age,profession,zodiac_sign,date,blog_content,number_of_posts
0,1000331,female,37,indUnk,Leo,2004-05-31,"Well, everyone got up and going this morning. ...",1
1,1000331,female,37,indUnk,Leo,2004-05-29,My four-year old never stops talking. She'll ...,1
2,1000331,female,37,indUnk,Leo,2004-05-28,"Actually it's not raining yet, but I bought 15...",4
3,1000331,female,37,indUnk,Leo,2004-06-21,My 20th high school urlLink reunion is this ...,1
4,1000331,female,37,indUnk,Leo,2004-06-18,We always have pizza on Friday nights. It tak...,1
...,...,...,...,...,...,...,...,...
331396,999503,male,25,Internet,Cancer,2004-07-05,"Chillin to some groove salad, studying BGP con...",1
331397,999503,male,25,Internet,Cancer,2004-07-04,Today we celebrate our independence day. In...,1
331398,999503,male,25,Internet,Cancer,2004-07-03,"Ugh, I think I have allergies... My nose has ...",1
331399,999503,male,25,Internet,Cancer,2004-07-02,"""Science is like sex; occasionally something p...",1


In [109]:
body_bac = df_bac[['blog_content']]

Conversión de texto del archivo BAC como strings para evitar errores en futuros pasos

In [118]:
body_bac['blog_content'] = body_bac[['blog_content']].astype(str)

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/3623974741.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['blog_content'] = body_bac[['blog_content']].astype(str)


In [119]:
#Conjunto de oraciones
body_bac

,blog_content
0,"Well, everyone got up and going this morning. ..."
1,My four-year old never stops talking. She'll ...
2,"Actually it's not raining yet, but I bought 15..."
3,My 20th high school urlLink reunion is this ...
4,We always have pizza on Friday nights. It tak...
...,...
331396,"Chillin to some groove salad, studying BGP con..."
331397,Today we celebrate our independence day. In...
331398,"Ugh, I think I have allergies... My nose has ..."
331399,"""Science is like sex; occasionally something p..."


Se aplica la misma función de normalización que se utilizó para datos 20N

In [120]:
body_bac['normalized'] = body_bac['blog_content'].apply(normalize_text)

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/2090022200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['normalized'] = body_bac['blog_content'].apply(normalize_text)


Se aplica word_tokenize de la librería nltk

In [122]:
body_bac['tokenized'] = body_bac['normalized'].apply(word_tokenize)

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/1310342155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['tokenized'] = body_bac['normalized'].apply(word_tokenize)


Se aplica el reemplazo de palabras a dígitos

In [125]:
body_bac['digits'] = body_bac['tokenized'].apply(lambda tokens: [words_to_numbers(token) for token in tokens])

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/757422033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['digits'] = body_bac['tokenized'].apply(lambda tokens: [words_to_numbers(token) for token in tokens])


Se aplica el reemplazo de dígitos al caracter NUM

In [126]:
body_bac['replacement'] = body_bac['digits'].apply(lambda tokens: [replace_digits(token) for token in tokens])

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/1313792504.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['replacement'] = body_bac['digits'].apply(lambda tokens: [replace_digits(token) for token in tokens])


Se añade <s> al comienzo de cada frase y </s> al término de cada frase

In [127]:
body_bac['sentence'] = body_bac['replacement'].apply(lambda tokens: ['<s>'] + tokens + ['</s>'])

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/4251335818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['sentence'] = body_bac['replacement'].apply(lambda tokens: ['<s>'] + tokens + ['</s>'])


Se aplica el mismo proceso para el reemplazo de tokens únicos en todo el corpus

In [128]:
all_tokens = [token for tokens in body_bac['sentence'] for token in tokens]
token_freq = Counter(all_tokens)

In [131]:
body_bac['unique'] = body_bac['sentence'].apply(lambda tokens: replace_unique_tokens(tokens, token_freq))

/var/folders/j2/dt9fh71s1yq44ph9g2dysl4r0000gn/T/ipykernel_77944/462386146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  body_bac['unique'] = body_bac['sentence'].apply(lambda tokens: replace_unique_tokens(tokens, token_freq))


Se exportan los datos a un dataframe único en un csv

In [132]:
results_bac = pd.DataFrame(body_bac['unique'].apply(lambda tokens: ' '.join(tokens)))

In [133]:
results_bac.to_csv("bac_tokenized.csv",index=False)

# DIVISIÓN DE DATOS

Se utilizó la librería train_test_split de scikit-learn para realizar la división aleatoria de datos, se hizo la división correspondiente de 80% para entrenamiento y 20% para prueba para ambos archivos

## DIVISIÓN DE DATOS - 20N

In [ ]:
#lectura de archivo tokenizadoo
df_20n = pd.read_csv("20n_tokenized.csv")

In [ ]:
#División de datos 80% - 20%
train_20n, test_20n = train_test_split(df_20n, train_size=0.8, random_state=42)

Se exportan los datos obtenidos de la división a un csv

In [ ]:
train_20n.to_csv("20N_01_training.csv")
test_20n.to_csv("20N_01_testing.csv")

## DIVISIÓN DE DATOS - BAC

In [ ]:
#lectura de archivoo
df_bac = pd.read_csv("bac_tokenized.csv")

In [ ]:
#División de datos 80% - 20%
train_bac, test_bac = train_test_split(df_bac, train_size=0.8, random_state=42)

Se exportan los datos obtenidos de la división a un csv

In [ ]:
train_bac.to_csv("BAC_01_training.csv")
test_bac.to_csv("BAC_01_testing.csv")